In [50]:
import pandas as pd
import numpy as np

import datetime


In [51]:
# TODO: change to parquet
PRICES_INPUT_FILE = "../data/raw/prices.csv"
PROCESSED_INPUT_FILE = "../data/processed/products_clustering.parquet"

In [52]:
prices = pd.read_csv(PRICES_INPUT_FILE)
prices.head()

,date,price,price_plus,game_id
0,2022-04-28,39.99,39.99,5154444
1,2022-04-28,69.99,69.99,4928471
2,2022-04-28,19.99,19.99,5198377
3,2022-04-28,59.99,59.99,5048054
4,2022-02-04,69.99,62.99,5065097


In [53]:
products = pd.read_parquet(PROCESSED_INPUT_FILE)
products.head()

,id,subscribers_count,float_price,days_since_release,days_since_last_update,is_ps5,is_ps4,is_ps3,is_psv,platforms_count,has_platinum
0,5154444,16,39.99,NaN,353.0,0,1,0,0,7,0
1,4928471,51,69.99,NaN,500.0,1,1,0,0,14,1
2,5198377,12,19.99,346.0,346.0,0,1,0,0,7,0
3,5048054,13,59.99,NaN,423.0,1,1,0,0,14,1
4,5065097,13,69.99,NaN,359.0,1,0,0,0,7,1


In [54]:
# Fix data types
prices["date"] = pd.to_datetime(prices["date"])

In [55]:
prices["previous_price"] = prices.groupby(["game_id"])["price"].shift()
prices["price_change"] = (prices["previous_price"] - prices["price"]) / prices[
    "previous_price"
]
prices["is_discount"] = (prices["price_change"] > 0.0).astype(int)


In [56]:
prices["previous_date"] = prices.groupby("game_id")["date"].shift(1)

prices["duration"] = prices["date"] - prices["previous_date"]
prices["duration"] = [x.days for x in prices["duration"]]


In [57]:
LOOK_BACK_RANGE = [1, 7, 14, 21, 30, 60, 90, 120]
LOOK_AHEAD_RANGE = [1, 7, 14, 21, 30]

ts = prices[["game_id", "price"]].copy()

for i in LOOK_BACK_RANGE:
    ts[f"price_{i}_days_back"] = ts.groupby("game_id")["price"].shift(i)

for i in LOOK_AHEAD_RANGE:
    ts[f"price_{i}_days_ahead"] = ts.groupby("game_id")["price"].shift(-i)


In [58]:
# Aggregations and statistics from prices dataset

copy = products.copy()

copy["average"] = prices.groupby("game_id").agg({"duration": "mean"})

In [59]:
prices.columns

Index(['date', 'price', 'price_plus', 'game_id', 'previous_price',
       'price_change', 'is_discount', 'previous_date', 'duration'],
      dtype='object')

In [60]:
discounts = prices.query("is_discount == 1")
discounts.head()

,date,price,price_plus,game_id,previous_price,price_change,is_discount,previous_date,duration
11,2021-11-19,45.49,45.49,4672196,69.99,0.350050,1,2021-06-14,158.0
13,2021-12-09,45.49,45.49,4672196,69.99,0.350050,1,2021-11-30,9.0
15,2021-12-22,45.49,45.49,4672196,69.99,0.350050,1,2021-12-13,9.0
17,2022-02-02,41.99,41.99,4672196,69.99,0.400057,1,2022-01-08,25.0
19,2022-03-16,34.99,34.99,4672196,69.99,0.500071,1,2022-02-10,34.0


In [61]:
stats = discounts.groupby("game_id").agg(
    average_discount_duration=("duration", np.mean),
    average_discount_amount=("price_change", np.mean),
    average_discounted_price=("price", np.mean),
    minimum_price=("price", min),
    times_discounted=("price", np.count_nonzero),
)
stats.head()


,average_discount_duration,average_discount_amount,average_discounted_price,minimum_price,times_discounted
game_id,,,,,
39149,95.100000,0.665911,5.008000,2.99,20
39158,149.466667,0.614365,5.780667,2.84,15
39179,304.250000,0.550451,3.992500,1.99,4
39181,176.625000,0.562171,5.818750,2.99,8
39187,284.555556,0.729227,4.058889,2.99,9


In [62]:
# TODO: Time based and sliding window aggregations


In [64]:
products.head()

,id,subscribers_count,float_price,days_since_release,days_since_last_update,is_ps5,is_ps4,is_ps3,is_psv,platforms_count,has_platinum
0,5154444,16,39.99,NaN,353.0,0,1,0,0,7,0
1,4928471,51,69.99,NaN,500.0,1,1,0,0,14,1
2,5198377,12,19.99,346.0,346.0,0,1,0,0,7,0
3,5048054,13,59.99,NaN,423.0,1,1,0,0,14,1
4,5065097,13,69.99,NaN,359.0,1,0,0,0,7,1


In [66]:
products = products.set_index("id")

In [67]:
# TODO
ts = ts.set_index("game_id")

In [73]:
outfile = products.join(stats, how="left").reset_index()

In [74]:
outfile.head()

,id,subscribers_count,float_price,days_since_release,days_since_last_update,is_ps5,is_ps4,is_ps3,is_psv,platforms_count,has_platinum,average_discount_duration,average_discount_amount,average_discounted_price,minimum_price,times_discounted
0,5154444,16,39.99,NaN,353.0,0,1,0,0,7,0,NaN,NaN,NaN,NaN,NaN
1,4928471,51,69.99,NaN,500.0,1,1,0,0,14,1,NaN,NaN,NaN,NaN,NaN
2,5198377,12,19.99,346.0,346.0,0,1,0,0,7,0,NaN,NaN,NaN,NaN,NaN
3,5048054,13,59.99,NaN,423.0,1,1,0,0,14,1,NaN,NaN,NaN,NaN,NaN
4,5065097,13,69.99,NaN,359.0,1,0,0,0,7,1,NaN,NaN,NaN,NaN,NaN


Example table output schema:
```
SELECT
    id,
    subscribers_count,
    current_price,
    is_ps5,
    is_ps4,
    is_ps3,
    is_psv,
    platforms_count,
    times_discounted,
    average_discount,
    average_duration,
    average_price,
    days_since_release,
    days_since_last_update,
FROM
    db.table;
```

In [69]:
# If there are NaNs in the final output, drop them
# outfile = outfile.dropna()
# outfile.isna().sum()

In [75]:
outfile.to_parquet("../data/processed/product_statistics.parquet")